<a href="https://colab.research.google.com/github/jjspergel/EarthEngine/blob/master/first_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import ee
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/vAGEJD-scjial3ZOlv-LFJH5SF2bFOVOeFooDcwD68zUNKCLcFwuo5U



Successfully saved authorization token.


In [27]:
import numpy as np
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import cartopy.crs as ccrs
import cartopy
import shapefile

In [28]:
#Import shapefiles


In [29]:
d = pd.read_csv('scripps_iceshelves.csv')
geo = d['.geo']
poly_list = []
#convert the list of string coordinates into a list of float -> ee.Polygon
for poly in np.arange(0,len(geo)):
    test_poly = geo[poly][35:-4].rsplit('],[');
    coord_list = []
    for substring in test_poly:
        [lon,lat] = substring.split(',')
        coord_list.append([float(lon.strip('[]')),float(lat.strip('[]'))])
    polygon = ee.Geometry.Polygon(coord_list)
    poly_list.append(polygon)
    

In [30]:
scripps_iceshelves = ee.FeatureCollection(poly_list)
#Individual ice shelves
amery = scripps_iceshelves.select([70]).geometry()

#Change here to pick different ice shelves
geometry_iceshelves = amery;

In [31]:
#Import Summer Landsat images (1-31 DOY, between 2015-1-1 to 2019-2-1, cloud cover <30%)
LandsatSummers = ee.ImageCollection('LANDSAT/LC08/C01/T2_TOA').select(['B2','B3','B4','B6','B8','B10']).filterDate('2015-01-01','2019-02-01').filterBounds(geometry_iceshelves).filter(ee.Filter.calendarRange(1,31,'day_of_year')).filterMetadata('CLOUD_COVER','less_than',0.3)

In [32]:
def makeLakeMask(image):
  #lake thresholds
    NDWI = image.normalizedDifference(['B2','B4']).gte(0.19)
    B2 = image.select('B2');
    B3 = image.select('B3');
    B4 = image.select('B4');
    B3B4difference = B3.subtract(B4).gte(0.07)
    B2B3difference = B2.subtract(B3).gte(0.07)
    lakethreshold = NDWI.bitwise_and(B3B4difference).bitwise_and(B2B3difference)

  #cloud mask
    B6 = image.select('B6')
    NDSI = image.normalizedDifference(['B3','B6'])
    NDSImask = NDSI.lt(0.8)
    B6mask = B6.gte(0.1)
    B2mask = B2.gte(0.6).bitwise_and(B2.lte(0.95))
    cloudmask = NDSImask.bitwise_and(B6mask).bitwise_and(B2mask)
  
  #rock mask
    B10 = image.select('B10');
    TIRSBlue = B10.divide(B2)
    TIRSBluemask = TIRSBlue.gte(0.65)
    B2mask = B2.lt(0.35)
    B4mask = B4.gt(0)
    rockmask = TIRSBluemask.bitwise_and(B2mask).bitwise_and(B4mask)

    LakeMask = lakethreshold.subtract(rockmask).subtract(cloudmask)
    return LakeMask


In [33]:
LakeMasks_LandsatSummers = LandsatSummers.map(makeLakeMask);
maxlakethresholdmap = LakeMasks_LandsatSummers.reduce('max')

Moussavi_lakes = maxlakethresholdmap.gt(0).clip(geometry_iceshelves)

In [34]:
#REMA import

REMAmosaic = ee.Image('UMN/PGC/REMA/V1_1/8m').select('elevation').divide(.001).clip(geometry_iceshelves).toInt64();

In [35]:
# Pick out Sinks
sinks = ee.Terrain.fillMinima(REMAmosaic,
                              ee.Number(30)).clip(geometry_iceshelves)

           
filledSinks = sinks.subtract(REMAmosaic);

EEException: Request payload size exceeds the limit: 4194304 bytes.

In [39]:
both_L8_REMAfilled = filledSinks.bitwise_and(Moussavi_lakes);
only_L8 = Moussavi_lakes.bitwise_and(filledSinks.bitwise_not())
only_REMA = filledSinks.bitwise_and(Moussavi_lakes.bitwise_not())



EEException: Request payload size exceeds the limit: 4194304 bytes.